#Welcome to Berlin Baby ! 

![picture](https://drive.google.com/uc?id=1BiFxL5vcp__WYZl9ZzBdkwjfv_xbc5mF)






# Introduction

The target is to provide some technical insights about geomarketing using python
- How to calculate an attractivity score of a series of sample points (red star)  placed randomly in a map.
- Decompose the steps visually 
- Adapt, improve and update a model. 


The attractivity will be based on the resident population of the neighborhoods around each sample, divided by the other players in each of these neighborhood 
- It is a base case , thus kept as simple as possible. 


For this we need the following ingredients : 
- shapes of neighborhoods,  
- the population in these shapes, 
- geolocalized list of competitors,
- optionnally a 'quality' of each point of interest.

#Detailed Steps : 

#1- Size of the cakes 
  - We choose a city or an area of interest,
  - We create and a bounding rectangle around it,
  - We place (p) points chosen randomly within the boundaries of the rectangle,

  - From each of these points we create a circle of radius (r),
  - We load national open datsets : districts shapes and their population attribute
  - We intersect the circle with the different neigbourhoods,
  - We collect the population in each of the neighborhood

#2- Other players in the area
  - We calculate the centroid of each neighborhood
  - we calculate the distance from the centroid to the sample
  - we create another buffer circle around each centroid with the radius equal to the distance from centroid to sample. 
  - we collect all other competitors within this circle

#3- Scoring of each sample  

- Given a gravity formula , we sill score each sample 
- We save the hypothesis and the results of our sudies 

#4- To do : Calibrate the model : example of buffer sizes

- We arbitrarily set a value of the buffer size but how relevant is it ? 

- To do : we need some points where we know the real number of customers per a given period of time; ie the real score
- We need to conduct many simulations and change the parameters values;  for instance the bufffer sizes 
- we will observe the error with regards to the buffer size 
- we will aim at finding the best buffer size


#Credits 

Credits should be given to : 
- Juan Orduz for very good code for python geoprocessing,
 and the many open data links that were also used here:
"Open Data: Germany Maps Viz"
https://juanitorduz.github.io/germany_plots/

- GeoPandas developers
https://geopandas.org/docs/user_guide/geometric_manipulations.html

- mplleaflet developpers 
https://github.com/jwass/mplleaflet/graphs/contributors

- Overpass API that enables easy download of data
https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_API_by_Example

- Open street map and its contributors

# Licence : 
- to do / open licence but mention your source

# coordinates reference systen used : 

- epsg:4326 


In [2]:
#Libraries not present in colab or with more complex dependencies
!pip install fiona shapely pyproj rtree &> /dev/null
!pip install geopandas &> /dev/null
!pip install mplleaflet &> /dev/null
#https://stackoverflow.com/questions/66811372/geopandas-overlay-function-issue-spatial-indexes-require-either-rtree-or-pyg



#libraries that need to be imported in colab
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point, LineString
import geopandas as gpd
import pandas as pd
pd.options.mode.chained_assignment = None 


import mplleaflet
import zipfile
from datetime import datetime

In [3]:


class attraction(): 
  ''' Generate points and shapes for the simulations'''

  def generate_rectangle(rect_center: list, x_range: float = 0.1 , y_range :float = 0.05)-> dict: 
    '''We genereate a rectangle around the point '''
    limits = {'xmin' : rect_center[0]-x_range, 'xmax' : rect_center[0]+ x_range, 
              'ymin' : rect_center[1]-y_range,  'ymax' : rect_center[1]+ y_range}
    return limits

  def convert_to_gdf(limits: dict, crs : str)-> gpd.GeoDataFrame(): 
    '''We generate a one line geodataframe to facilitate geo intersection with other gdfs.
    Limits must possess xmin, xmax, ymin, ymax keys''' 
    polys1 = gpd.GeoSeries([Polygon([(limits['xmin'],limits['ymin']), (limits['xmax'],limits['ymin']), 
                                  (limits['xmax'],limits['ymax']), (limits['xmin'],limits['ymax'])])])
    gpd_ = gpd.GeoDataFrame({'geometry': polys1, 'not_used':[1]}, crs=crs)

    return gpd_

  def generate_points(limits: dict , crs : int , points : int = 3 )-> gpd.GeoDataFrame(): 
    '''We genereate a random number of points in the area defined in limits'''
    xc = (limits['xmax'] - limits['xmin']) * np.random.random(points) + limits['xmin']
    yc = (limits['ymax'] - limits['ymin']) * np.random.random(points) + limits['ymin']

    points_list = [Point(x, y) for x, y in zip(xc, yc)]
    default_index = [i for i in range(len(points_list))]

    gpd_ = gpd.GeoDataFrame({'nameofsample': default_index, 'geometry' : points_list,} , crs=crs)

    return gpd_

  def generate_buffer(gpd_: gpd.GeoDataFrame() , distance: float = 0.01 , resolution: int = 4 , crs : int = 4326):
    '''we  generate a buffer based on distance '''
    gpd_['buffer'] = gpd_["geometry"].buffer(distance = distance, resolution = 4).to_crs(crs)
    gpd_.rename(columns={'geometry':'geom_points', 'buffer':'geometry'}, inplace=True) #buffer as geometry

    return gpd_

  def generate_buffers(gdf_ : gpd.GeoDataFrame, point1 : str = 'sample_point', point2 : str = 'neighborhood_centroid' ) -> gpd.GeoDataFrame : 
    "Generate buffers with distances from sample to centroids using geopandas"
    gdf_['distances'] = gdf_[point1].distance(gdf_[point2], align=True)
    print('ok')
    gdf_['buffers'] = gdf_[point2].buffer(distance = gdf_['distances'], resolution = 4) #.to_crs(crs)

    return gdf_


  

######



class prepareShapes:
    def __init__(self, data_sources_paths: dict ):
        self.data_sources_paths = data_sources_paths

    def load_gdf(self, key: str, joining_col : str )-> gpd.GeoDataFrame :
      """helper to load a geodataframe - works with geojson and shapefiles"""
      gdf = gpd.read_file(self.data_sources_paths[key], 
                              dtype={joining_col: str})
      return gdf

    def load_df(self, key: str, joining_col : str, zipped : bool = False) -> pd.DataFrame :
        """Enables to load the attributes dataframe specific to France & Germany"""
        if not zipped : 
          df = pd.read_csv(self.data_sources_paths[key], 
                                    sep=',', dtype={joining_col: str})
        else :
          zf = zipfile.ZipFile(self.data_sources_paths[key])

        return df

  

class mergeShapes:
  """class to regroup geographical joins
  """

  def prepare_osm_gdf(gdf: gpd.GeoDataFrame, keep_cols: list ) -> gpd.GeoDataFrame : 
      """Prepare the geodataset that enables to have competition:  filter colums, identify own installations, and creating a joining col"""

      #1- filter colums : keep only columns that are not too empty
      gdf_ = gdf[keep_cols] 

      #2- try to identify Total stations based on two other columns
      gdf_["brand"] = gdf_["brand"].fillna("Unknown")
      gdf_["name"] = gdf_["name"].fillna("Unknown")
      
      gdf_['is_mine_brand'] = gdf_["brand"].str.contains("total", case=False) 
      gdf_['is_mine_name'] = gdf_["name"].str.contains("total", case=False)
      gdf_['is_mine'] = (gdf_['is_mine_brand']) | (gdf_['is_mine_name'])
      gdf_.drop(['is_mine_brand', 'is_mine_name'], axis=1, inplace=True)

      #3- copy the geometry for later geographical joins
      gdf_['node_coord'] = gdf_['geometry'] 

      return gdf_


  def geofilter_rectangle(gdf_bbox, gdf_comp) : 
      '''geofilters to the bounding box and we rename a new geometry columns for osm points'''
      gdf_ = gpd.sjoin(gdf_bbox, gdf_comp, how="left", op='intersects')
      gdf_.drop(['geometry', 'index_right'], axis=1, inplace=True)


      return gdf_


  def geofilter_plz(gdf, gdf2, osm_cols) : 
      """geofilters competition within the plz shapes"""
      gdf = gpd.sjoin(gdf, gdf2, how="inner", op='intersects')
      gdf.drop(['index_right', 'not_used'], axis=1, inplace=True)

      gdf = gdf[osm_cols]
      return gdf


  def geofilter_buffers(gpd1, gpd2, osm_cols): 
    """geofilters competition within buffer circle
    gpd1 contains the buffers gpd2 contains the points"""

    gpd2['competitor_geom'] = gpd2['geometry']

    gpd1 = gpd1.loc[:, ["buffers",  "nameofsample", "plz"] ]
    gpd1.rename(columns={'buffers':  'geometry'}, inplace=True) 

    gpd_ = gpd.sjoin(gpd1, gpd2, how="inner", op='intersects')

    gpd_ = gpd_[osm_cols]

    return gpd_


#Class to standardize visualisations 
class visualize:
  """used to have homogenuous visuals"""

  def plot_samples_points(gdf : gpd.GeoDataFrame, geom_col: str):
    """samples layers"""
    gdf.set_geometry(geom_col).plot(ax=ax,  marker = "*", color = "red")
    

  def plot_samples_buffers(gdf : gpd.GeoDataFrame, geom_col: str):
    """buffers arount sample points"""
    gdf.set_geometry(geom_col).plot(ax=ax, color='yellow', edgecolor='blue')
    

  def plot_neighborhoods(gdf : gpd.GeoDataFrame, geom_col: str):
    """neighbouring attributes layers"""
    gdf.set_geometry(geom_col).plot(ax=ax, color='blue', edgecolor='black')


  def plot_centroids(gdf : gpd.GeoDataFrame, geom_col: 'str' = 'centroids'):
    """neighbouring centroids"""
    gdf.set_geometry(geom_col).plot(ax=ax, color='black', edgecolor='black')


  def plot_buffers(gdf : gpd.GeoDataFrame, geom_col: 'str' = 'buffers'):
    """neighbouring attributes layers""" 
    gdf.set_geometry(geom_col).plot(ax=ax,  edgecolor='green')


  def plot_competition(gdf : gpd.GeoDataFrame, geom_col: 'str' = 'node_coord' ):
    """plot competition, is_mine is a boolean if the poi has a brand keyword in its name"""
    gdf.set_geometry(geom_col).plot(ax=ax, column='is_mine', categorical=True, edgecolor='black')


  def plot_lines(gdf : gpd.GeoDataFrame, geom_col: 'str' = 'line_to_sample' ):
    """Plot lines geography"""
    gdf.set_geometry(geom_col).plot(ax=ax, color = 'black')

  def plot_winner(gdf : gpd.GeoDataFrame, geom_col: str):
    """winner"""
    gdf.set_geometry(geom_col).plot(ax=ax,  marker = "*", color = "blue")


In [4]:
#parameters

crs = 4326 
# 4326 is degrees , 3035 is metres
#about projections
#https://stackoverflow.com/questions/63004400/getting-a-userwarning-when-calculating-centroid-of-a-geoseries/63038899#63038899

top_cities = {
    'Berlin': (13.404954, 52.520008), 
    'Cologne': (6.953101, 50.935173),
    'Düsseldorf': (6.782048, 51.227144),
    'Frankfurt am Main': (8.682127, 50.110924),
    'Hamburg': (9.993682, 53.551086),
    'Leipzig': (12.387772, 51.343479),
    'Munich': (11.576124, 48.137154),
    'Dortmund': (7.468554, 51.513400),
    'Stuttgart': (9.181332, 48.777128),
    'Nuremberg': (11.077438, 49.449820),
    'Hannover': (9.73322, 52.37052),
    'LaRochelle': (-1.151139, 46.160329),
    'Lyon' : (4.8400, 45.7600),
    'Kiel' : (9.985595 , 54.3417317)
}

countries = {
    'Berlin': "DE", 
    'Cologne': "DE",
    'Düsseldorf': "DE",
    'Frankfurt am Main': "DE",
    'Hamburg': "DE",
    'Leipzig': "DE",
    'Munich':  "DE",
    'Dortmund': "DE",
    'Stuttgart': "DE",
    'Nuremberg': "DE",
    'Hannover': "DE",
    'LaRochelle': "FR", 
    'Lyon' : "FR"
}


#we have to change data based on the country
country_col_name = {"FR" : "IRIS", "DE" : "plz", "UK" : 'lad17cd' }


#we have to change data based on the country
local_context = {
                  "FR" : 
                  { "local_attributes" : {"population": " ", "shapes" : "IRIS"}, 
                    "data_sources" : 
                      {'drive' : 
                              {
                                "populationDataset_fr" : "/content/drive/MyDrive/Colab Notebooks/data/GIS/France/base-ic-evol-struct-pop-2017_csv.zip",
                                "shapes_IRIS_2013" : "/content/drive/MyDrive/Colab Notebooks/data/GIS/France/iris-2013-01-01.zip",
                                "overpass_fuel_fr": "/content/drive/MyDrive/Colab Notebooks/data/GIS/France/overpasFR_Fuel.geojson",
                                "chargingstations_fr" : "/content/drive/MyDrive/Colab Notebooks/data/GIS/France/overpass_charging_station_westeur.geojson"
                                }} },
                 
                "DE" : "plz", 
                "UK" : "" }


#story : only use priorietary tools (esri) when non possible otherwise

#cities coordinates:
city_loc_link = '/content/drive/MyDrive/Colab Notebooks/data/GIS/Metadata/cities500.zip'

#cities: 
#https://download.geonames.org/export/dump/





#############


#Data sources 


#Germany
data_sources_paths_de = {
    "shapes" : "/content/drive/MyDrive/Colab Notebooks/data/GIS/Germany/plz-gebiete.shp.zip",
    "placesNames" : "/content/drive/MyDrive/Colab Notebooks/data/GIS/Germany/zuordnung_plz_ort.csv",
    "populationDataset" : "/content/drive/MyDrive/Colab Notebooks/data/GIS/Germany/plz_einwohner.csv",
    "poiData": "/content/drive/MyDrive/Colab Notebooks/data/GIS/Germany/total_de_stations.csv",
    "poiAttributes": "/content/drive/MyDrive/Colab Notebooks/data/GIS/Germany/sqldsv_turnover_berlin_daily2106.csv",
    "CompetitionData": "/content/drive/MyDrive/Colab Notebooks/data/GIS/Germany/germanyoverpass.geojson",
    "chargingstations_de": "/content/drive/MyDrive/Colab Notebooks/data/GIS/Germany/overpass_charging_stations_de.geojson",
    "sampleData" : "/content/drive/MyDrive/Colab Notebooks/data/GIS/sample.geojson"
    }

#France
data_sources_paths_fr = {
    "populationDataset_fr" : "/content/drive/MyDrive/Colab Notebooks/data/GIS/France/base-ic-evol-struct-pop-2017_csv.zip",
    "shapes_IRIS_2013" : "/content/drive/MyDrive/Colab Notebooks/data/GIS/France/iris-2013-01-01.zip",
    "overpass_fuel_fr": "/content/drive/MyDrive/Colab Notebooks/data/GIS/France/overpasFR_Fuel.geojson",
    "chargingstations_fr" : "/content/drive/MyDrive/Colab Notebooks/data/GIS/France/overpass_charging_station_westeur.geojson"
    }
###
logs_path = '/content/drive/MyDrive/Colab Notebooks/data/GIS/scores_histo.csv'



In [5]:
#from now we need to get external data. Here it is stored in google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:

#for later use
#cols = ['geonameid'  ,'name' , 'asciiname' , 'alternatenames' , 'latitude' , 'longitude' , 
#        'fclass' , 'fcode' , 'country' ,'cc2' , 'admin1' , 'admin2' , 'admin3' , 'admin4' , 
#        'population' , 'elevation' , 'gtopo30' , 'timezone' , 'moddate' ]

#df = pd.read_csv(city_loc_link, sep='\t', header = None )
#df.columns = cols
#df.head()

#1- Size of the cakes

We choose a city or an area of interest

In [7]:
#We choose a city or a place of interest.
country = 'DE'
city = 'Berlin'
figsize = (5,7) 
points = 2 #number of samples 
radius = 0.02 #buffers size

We create and a bounding rectangle around it

We place the (p) points chosen randomly within the boundaries of the rectangle

From each of these points we create a circle of radius (r)

In [8]:
#Generate samples geodataframe (geometry set on buffer polygons)

#We create and a bounding rectangle around it.
limits_dict = attraction.generate_rectangle(top_cities[city], x_range = 0.15 , y_range = 0.055)

#We place (p) points chosen randomly within the boundaries of the rectangle.
samples_gdf = attraction.generate_points(limits_dict, crs = crs, points  = points )

#From each of these points we create a buffer circle of radius (r) 
samples_gdf = attraction.generate_buffer(samples_gdf, distance = radius)

samples_gdf.rename(columns={'geometry':'sample_buffer', 'geom_points':'sample_point'}, inplace=True)


##################
#visualize Samples
fig, ax = plt.subplots(figsize = figsize)

visualize.plot_samples_points(samples_gdf, 'sample_point')
visualize.plot_samples_buffers(samples_gdf, 'sample_buffer')

mplleaflet.display(fig=fig)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




  - We load national open datsets : districts shapes and their population attribute and we merge them


In [9]:
#We prepare country geodataframe

#Instanciation
shapes = prepareShapes(data_sources_paths_de)

#load shapes of neighborhoods
shape_gdf = shapes.load_gdf('shapes', country_col_name[country]) 
#load additional placesNames attribute( note, plz are not unique)
region_df = shapes.load_df('placesNames', country_col_name[country]) 

# Specific to Germany : Add region attribute to shapes using village.
country_gdf = pd.merge( left=shape_gdf, right=region_df, on=country_col_name[country],how='inner')
country_gdf.drop(['osm_id', 'ort'], axis=1, inplace=True)

#add population data
#load populationDataset by neighbourhood
pop_df = shapes.load_df('populationDataset', country_col_name[country]) 

#We merge shapes and population by district
country_gdf = pd.merge(left=country_gdf, right=pop_df, on=country_col_name[country],how='left')

#free memory
del(shape_gdf, region_df, pop_df)

  - We intersect the circle with the different neigbourhoods,
  - We collect the population in each of the neighborhood

In [10]:
country_gdf

,plz,note,geometry,bundesland,einwohner
0,52538,"52538 Gangelt, Selfkant","POLYGON ((5.86632 51.05110, 5.86692 51.05124, ...",Nordrhein-Westfalen,21390
1,52538,"52538 Gangelt, Selfkant","POLYGON ((5.86632 51.05110, 5.86692 51.05124, ...",Nordrhein-Westfalen,21390
2,47559,47559 Kranenburg,"POLYGON ((5.94504 51.82354, 5.94580 51.82409, ...",Nordrhein-Westfalen,10220
3,52525,"52525 Waldfeucht, Heinsberg","POLYGON ((5.96811 51.05556, 5.96951 51.05660, ...",Nordrhein-Westfalen,49737
4,52525,"52525 Waldfeucht, Heinsberg","POLYGON ((5.96811 51.05556, 5.96951 51.05660, ...",Nordrhein-Westfalen,49737
...,...,...,...,...,...
15303,02899,"02899 Ostritz, SchÃ¶nau-Berzdorf","POLYGON ((14.85296 51.06854, 14.85449 51.06859...",Sachsen,4107
15304,02929,02929 Rothenburg/O.L.,"POLYGON ((14.85491 51.32895, 14.85608 51.33004...",Sachsen,5110
15305,02827,02827 GÃ¶rlitz,"POLYGON ((14.91168 51.14243, 14.91571 51.14571...",Sachsen,17068
15306,02828,02828 GÃ¶rlitz,"POLYGON ((14.93413 51.16084, 14.93451 51.16123...",Sachsen,12016


### cake_gdf : districts touched by the samples buffers

In [11]:
#Spatial join neighborhoods with the sample points 
###################
###################
###################
cake_gdf = gpd.sjoin(country_gdf, samples_gdf.set_geometry('sample_buffer'), how="inner", op='intersects')
###################
###################
###################
cake_gdf.rename(columns={'geometry':'neighborhood_shape'}, inplace=True)
cake_gdf.drop(['index_right'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if self.run_code(code, result):


### Visual : districts touched by the samples buffers

In [12]:

#####################
#Vizualise influence areas
fig, ax = plt.subplots(figsize=figsize)

#sample layers
visualize.plot_samples_points(samples_gdf, 'sample_point')
visualize.plot_samples_buffers(samples_gdf, 'sample_buffer')

#geom attributes layers
visualize.plot_neighborhoods(cake_gdf, 'neighborhood_shape')
 
mplleaflet.display(fig=fig)


In [13]:
#Look at the data 
cake_gdf

,plz,note,neighborhood_shape,bundesland,einwohner,nameofsample,sample_point
14148,14193,14193 Berlin Grunewald,"POLYGON ((13.18660 52.48930, 13.18757 52.49123...",Berlin,15505,1,POINT (13.31464 52.49286)
14228,14057,14057 Berlin Charlottenburg,"POLYGON ((13.27333 52.50804, 13.27334 52.50839...",Berlin,12237,1,POINT (13.31464 52.49286)
14231,10711,10711 Berlin Halensee,"POLYGON ((13.27741 52.49838, 13.27781 52.49848...",Berlin,9794,1,POINT (13.31464 52.49286)
14235,14199,14199 Berlin Schmargendorf,"POLYGON ((13.27887 52.47502, 13.27903 52.47517...",Berlin,13657,1,POINT (13.31464 52.49286)
14262,10627,10627 Berlin Charlottenburg,"POLYGON ((13.29473 52.50644, 13.29507 52.50682...",Berlin,11179,1,POINT (13.31464 52.49286)
14263,10709,10709 Berlin Wilmersdorf,"POLYGON ((13.29480 52.49224, 13.29558 52.49264...",Berlin,8771,1,POINT (13.31464 52.49286)
14264,10629,10629 Berlin Charlottenburg,"POLYGON ((13.29487 52.50191, 13.29600 52.50225...",Berlin,11420,1,POINT (13.31464 52.49286)
14267,14197,14197 Berlin Wilmersdorf,"POLYGON ((13.29900 52.47353, 13.29908 52.47383...",Berlin,17889,1,POINT (13.31464 52.49286)
14268,10587,10587 Berlin Charlottenburg,"POLYGON ((13.29967 52.52112, 13.30012 52.52188...",Berlin,11069,1,POINT (13.31464 52.49286)
14269,10713,10713 Berlin Wilmersdorf,"POLYGON ((13.30033 52.48802, 13.30082 52.48833...",Berlin,10686,1,POINT (13.31464 52.49286)


##samples_gdf : size of the market of each sample

In [14]:
#size of the cakes for each sample
cake_description = cake_gdf.groupby(['nameofsample']).agg({'einwohner': 'sum','plz': 'count' })
cake_description = cake_description.reset_index()

#we enrich samples_gdf for later
samples_gdf = samples_gdf.merge(cake_description, on = 'nameofsample', how = 'left')
samples_gdf.rename(columns={'einwohner':'Tot_Population', 'plz':'Num_districts'}, inplace=True)
samples_gdf = gpd.GeoDataFrame(samples_gdf)
samples_gdf

,nameofsample,sample_point,sample_buffer,Tot_Population,Num_districts
0,0,POINT (13.50547 52.55331),"POLYGON ((13.52547 52.55331, 13.52395 52.54566...",145571,7
1,1,POINT (13.31464 52.49286),"POLYGON ((13.33464 52.49286, 13.33312 52.48521...",235812,21


#2- Other players in the area

We complete the districts of cake_gdf by a buffer. Their radius equal the distance from neigborhood centroid to the sample

In [15]:
#add centroids
cake_gdf['neighborhood_centroid'] = cake_gdf.set_geometry('neighborhood_shape').centroid

#add buffer circle around the centroids. the radius is based on distance from centroid to sample
cake_gdf = attraction.generate_buffers(cake_gdf, 'sample_point','neighborhood_centroid' )

#add a line between nodes and centroids
cake_gdf['line_to_sample'] = cake_gdf.apply(lambda x: LineString([x['neighborhood_centroid'], x['sample_point']]),axis=1)

ok


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [16]:
#####################
#Vizualise centroids and buffers
fig, ax = plt.subplots(figsize=figsize)

#sample layers
visualize.plot_samples_points(samples_gdf, 'sample_point')
#visualize.plot_samples_buffers(samples_gdf, 'sample_buffer')

#geom attributes layers
visualize.plot_neighborhoods(cake_gdf, 'neighborhood_shape')
visualize.plot_centroids(cake_gdf, 'neighborhood_centroid')
visualize.plot_buffers(cake_gdf, 'buffers')
visualize.plot_lines(cake_gdf, 'line_to_sample')

 
mplleaflet.display(fig=fig)

In [17]:
cake_gdf

,plz,note,neighborhood_shape,bundesland,einwohner,nameofsample,sample_point,neighborhood_centroid,distances,buffers,line_to_sample
14148,14193,14193 Berlin Grunewald,"POLYGON ((13.18660 52.48930, 13.18757 52.49123...",Berlin,15505,1,POINT (13.31464 52.49286),POINT (13.23653 52.48312),0.078710,"POLYGON ((13.31524 52.48312, 13.30925 52.45300...",LINESTRING (13.23653232323837 52.4831228440015...
14228,14057,14057 Berlin Charlottenburg,"POLYGON ((13.27333 52.50804, 13.27334 52.50839...",Berlin,12237,1,POINT (13.31464 52.49286),POINT (13.28794 52.50726),0.030329,"POLYGON ((13.31827 52.50726, 13.31596 52.49565...",LINESTRING (13.28794258841181 52.5072554468602...
14231,10711,10711 Berlin Halensee,"POLYGON ((13.27741 52.49838, 13.27781 52.49848...",Berlin,9794,1,POINT (13.31464 52.49286),POINT (13.29047 52.49812),0.024735,"POLYGON ((13.31520 52.49812, 13.31332 52.48866...",LINESTRING (13.29046880615491 52.4981231076173...
14235,14199,14199 Berlin Schmargendorf,"POLYGON ((13.27887 52.47502, 13.27903 52.47517...",Berlin,13657,1,POINT (13.31464 52.49286),POINT (13.29508 52.47766),0.024768,"POLYGON ((13.31985 52.47766, 13.31796 52.46819...",LINESTRING (13.29507950358791 52.4776644882777...
14262,10627,10627 Berlin Charlottenburg,"POLYGON ((13.29473 52.50644, 13.29507 52.50682...",Berlin,11179,1,POINT (13.31464 52.49286),POINT (13.30300 52.50799),0.019085,"POLYGON ((13.32209 52.50799, 13.32063 52.50068...",LINESTRING (13.30300014638788 52.5079872215610...
14263,10709,10709 Berlin Wilmersdorf,"POLYGON ((13.29480 52.49224, 13.29558 52.49264...",Berlin,8771,1,POINT (13.31464 52.49286),POINT (13.30306 52.49391),0.011620,"POLYGON ((13.31468 52.49391, 13.31380 52.48946...",LINESTRING (13.30306442901919 52.4939076340559...
14264,10629,10629 Berlin Charlottenburg,"POLYGON ((13.29487 52.50191, 13.29600 52.50225...",Berlin,11420,1,POINT (13.31464 52.49286),POINT (13.30855 52.50278),0.011637,"POLYGON ((13.32019 52.50278, 13.31930 52.49832...",LINESTRING (13.30854952155257 52.5027779043460...
14267,14197,14197 Berlin Wilmersdorf,"POLYGON ((13.29900 52.47353, 13.29908 52.47383...",Berlin,17889,1,POINT (13.31464 52.49286),POINT (13.31179 52.47336),0.019706,"POLYGON ((13.33150 52.47336, 13.33000 52.46582...",LINESTRING (13.31178952443845 52.4733619767482...
14268,10587,10587 Berlin Charlottenburg,"POLYGON ((13.29967 52.52112, 13.30012 52.52188...",Berlin,11069,1,POINT (13.31464 52.49286),POINT (13.31951 52.51847),0.026067,"POLYGON ((13.34558 52.51847, 13.34359 52.50849...",LINESTRING (13.31950948150534 52.5184684955012...
14269,10713,10713 Berlin Wilmersdorf,"POLYGON ((13.30033 52.48802, 13.30082 52.48833...",Berlin,10686,1,POINT (13.31464 52.49286),POINT (13.31328 52.48508),0.007898,"POLYGON ((13.32118 52.48508, 13.32058 52.48206...",LINESTRING (13.31327957042787 52.4850804709973...


In [18]:
#del(osm_gdf)

#load geojson sourced from OSM via http://overpass-turbo.eu/ [amenity=fuel] 

cond = 'osm_gdf' in dir()

if cond:  
   pass
else: 
  osm_gdf = shapes.load_gdf( "chargingstations_de", country_col_name[country])
  #osm_gdf = shapes.load_gdf( "CompetitionData", country_col_name[country])

  keep_cols = [ 'brand','name', 'addr:postcode', 'addr:housenumber', 'addr:street',  'geometry', 'source', 'id', 'website']

  osm_gdf = mergeShapes.prepare_osm_gdf(osm_gdf, keep_cols)


#we generate a gdf for the bounding box
gdf_bbox = attraction.convert_to_gdf(limits_dict, crs)


## bbox_osm_gdf : osm data filtered by a bounding box

In [19]:

#we generate a gdf for the bounding box

limits_dict_bigger = attraction.generate_rectangle(top_cities[city], x_range = 0.4 , y_range = 0.2)
gdf_bbox_bigger = attraction.convert_to_gdf(limits_dict_bigger, crs)

In [20]:
#we create a geofadaframe only for competition inside the bounding box using another spatial join 
########################
########################
bbox_osm_gdf = mergeShapes.geofilter_rectangle(gdf_bbox_bigger, osm_gdf) 

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [21]:
#we add a random quality to other players
#####################
#####################
bbox_osm_gdf['quality'] = np.random.uniform(1,2, len(bbox_osm_gdf))

In [22]:
#####################
#Vizualise centroids and buffers
fig, ax = plt.subplots(figsize=figsize)

#sample layers
visualize.plot_samples_points(samples_gdf, 'sample_point')

#geom attributes layers
visualize.plot_buffers(cake_gdf, 'buffers')

#other players layers

visualize.plot_competition(bbox_osm_gdf, 'node_coord')
 
mplleaflet.display(fig=fig)

## gdf_comp: other players in the neighbourhoods buffers

In [23]:
#filtering OSM points that are in a suburb

#we will want to keep the points coordinates
#bbox_osm_gdf['node_coord'] = bbox_osm_gdf['osm_poi_geom']

cake_gdf.set_geometry('buffers', inplace = True)
bbox_osm_gdf.set_geometry('node_coord', inplace = True)


#change the order of the dfs 
gdf_comp = gpd.sjoin(bbox_osm_gdf, cake_gdf,  how="inner", op='intersects')

gdf_comp_cols = ['nameofsample', 'sample_point', 'plz', 'einwohner', 'neighborhood_centroid', 'id', 'quality', 'is_mine', 
         'brand', 'name', 'node_coord']
#we lose the geodataframe format if we do not apply .loc ! 
gdf_comp= gdf_comp.loc[: , gdf_comp_cols]
gdf_comp.set_geometry('node_coord', inplace = True)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if self.run_code(code, result):


### Visual: other players

In [24]:
#####################
#Vizualise centroids and buffers
fig, ax = plt.subplots(figsize=figsize)

#sample layers
visualize.plot_samples_points(samples_gdf, 'sample_point')

#geom attributes layers
visualize.plot_centroids(cake_gdf, 'neighborhood_centroid')
visualize.plot_buffers(cake_gdf, 'buffers')

#other players layers
visualize.plot_competition(gdf_comp, 'node_coord')
 
mplleaflet.display(fig=fig)

#3- Scoring

#Huff Gravity Model Formula

![picture](https://drive.google.com/uc?id=1a-DnJrnW7T-6r9XrYlGiE3QAJ2EZPsMN)



source : A Probabilistic Analysis of Shopping Center Trade Areas
David L. Huff

In our context we will over simplify this algorithm : 
- Tij are all set to 1 , thus the formula becomes P(cij) = Sj / sum(Sj)

In [25]:
#samples are considered as one point in each neighborhood > we need to provide a quality
keep_cols = ['plz','einwohner', 'nameofsample', 'id', 'quality' , 'is_mine', 'brand', 'name', 'node_coord', 'sample_point']
cake_gdf.loc[:, ]

new_cake = cake_gdf.append(gdf_comp)
new_cake = new_cake.loc[:, keep_cols]

new_cake['quality'] = new_cake['quality'].fillna(value=1.5)

new_cake

,plz,einwohner,nameofsample,id,quality,is_mine,brand,name,node_coord,sample_point
14148,14193,15505,1,NaN,1.500000,NaN,NaN,NaN,None,POINT (13.31464 52.49286)
14228,14057,12237,1,NaN,1.500000,NaN,NaN,NaN,None,POINT (13.31464 52.49286)
14231,10711,9794,1,NaN,1.500000,NaN,NaN,NaN,None,POINT (13.31464 52.49286)
14235,14199,13657,1,NaN,1.500000,NaN,NaN,NaN,None,POINT (13.31464 52.49286)
14262,10627,11179,1,NaN,1.500000,NaN,NaN,NaN,None,POINT (13.31464 52.49286)
...,...,...,...,...,...,...,...,...,...,...
0,12161,16654,1,node/7918344319,1.448733,False,Unknown,Unknown,POINT (13.31772 52.45519),POINT (13.31464 52.49286)
0,12161,16654,1,node/8188176035,1.491647,False,Unknown,Unknown,POINT (13.31091 52.46039),POINT (13.31464 52.49286)
0,12161,16654,1,node/7056374801,1.458501,False,Unknown,Unknown,POINT (13.32465 52.46047),POINT (13.31464 52.49286)
0,12161,16654,1,node/7056382634,1.133587,False,Unknown,Unknown,POINT (13.32329 52.46049),POINT (13.31464 52.49286)


In [26]:
#each suburb has a given number of id(competitors) and we compute their average quality (which is wrong ! but it is just a poc here )
score_df = new_cake.groupby(['nameofsample', 'plz' ], as_index=False).agg({'id': 'count', 'einwohner': 'mean', 'quality': 'mean'})
score_df

,nameofsample,plz,id,einwohner,quality
0,0,10365,34,22665,1.471700
1,0,13051,0,21820,1.500000
2,0,13053,0,18486,1.500000
3,0,13055,3,27928,1.623227
4,0,13057,4,16106,1.489942
5,0,13059,1,14495,1.287553
6,0,13088,11,24071,1.510314
7,1,10587,54,11069,1.512501
8,1,10623,59,5680,1.502808
9,1,10625,35,10488,1.499678


In [27]:
#we multiply each row by the mean quality and divide by the number of competitors

########################
########################
score_df['plz_score'] = score_df['einwohner'] * ((score_df['quality']-1) /(1+score_df['id']))

########################
########################

score_df['plz_score'] = score_df['plz_score'].astype('int')
score_df

,nameofsample,plz,id,einwohner,quality,plz_score
0,0,10365,34,22665,1.471700,305
1,0,13051,0,21820,1.500000,10910
2,0,13053,0,18486,1.500000,9243
3,0,13055,3,27928,1.623227,4351
4,0,13057,4,16106,1.489942,1578
5,0,13059,1,14495,1.287553,2084
6,0,13088,11,24071,1.510314,1023
7,1,10587,54,11069,1.512501,103
8,1,10623,59,5680,1.502808,47
9,1,10625,35,10488,1.499678,145


In [28]:

#Scoring Algorithm : 
#for each sample xe aggregate each PLZ 

score_df_gb = score_df.groupby(['nameofsample'], as_index = False).agg({'plz': 'count' , 'einwohner' : 'sum', 'id': 'sum', 'plz_score': 'sum'})

#we add plenty of metadata for optimisation and reproduceability
score_df_gb['utc_time'] = datetime.today().strftime('%Y-%m-%d-%H:%M:%S')
score_df_gb['country'] = country
score_df_gb['city'] = city
score_df_gb['buffer_size'] = radius
score_df_gb['relative_comp_quality'] = 'random'
score_df_gb['algorithm'] = 'simple'
score_df_gb['dataset'] = data_sources_paths_de["CompetitionData"]

score_df_gb = score_df_gb.merge(samples_gdf, on = 'nameofsample' )

score_df_gb_gdf =  gpd.GeoDataFrame(score_df_gb, crs=crs, geometry = 'sample_point')

In [29]:
#we select the best sample
best = max(score_df_gb_gdf['plz_score'])
winner_ref = score_df_gb_gdf.loc[score_df_gb_gdf['plz_score'] ==  best,  ['nameofsample'] ].iloc[0,0]
winner_ref

0

In [30]:
#####################
#Vizualise centroids and buffers
fig, ax = plt.subplots(figsize=figsize)

#sample layers
visualize.plot_winner(samples_gdf.loc[samples_gdf['nameofsample'] == winner_ref, : ], 'sample_point')
visualize.plot_samples_points(samples_gdf.loc[samples_gdf['nameofsample'] != winner_ref, : ], 'sample_point')

#geom attributes layers

visualize.plot_neighborhoods(cake_gdf.loc[cake_gdf['nameofsample'] == winner_ref, : ], 'neighborhood_shape')

#visualize.plot_neighborhoods(cake_gdf, 'neighborhood_shape')
visualize.plot_centroids(cake_gdf.loc[cake_gdf['nameofsample'] == winner_ref, : ], 'neighborhood_centroid')
#visualize.plot_buffers(cake_gdf.loc[cake_gdf['nameofsample'] == winner_ref, : ], 'buffers')

#other players layers
if len(gdf_comp.loc[gdf_comp['nameofsample'] == winner_ref, : ]) > 0 :
    visualize.plot_competition(gdf_comp.loc[gdf_comp['nameofsample'] == winner_ref, : ], 'node_coord')
 
mplleaflet.display(fig=fig)


#The optimum location

In [31]:
#We can now generalize by saturating the map

In [32]:
#logs_path

######## Reinitialize / or Not ##############
#score_df_gb_gdf.to_parquet(logs_path)
####################################

gdfx = gpd.read_parquet(logs_path)
gdfx = gdfx.append(score_df_gb_gdf , ignore_index=True)
gdfx = gdfx.drop_duplicates()
gdfx.to_parquet(logs_path)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  # Remove the CWD from sys.path while we load stuff.


In [33]:
#filter by city an buffer size

gdfx = gdfx.loc[gdfx['city'] == city, : ]
gdfx = gdfx.loc[gdfx['buffer_size'] == radius, : ]

best = max(gdfx['plz_score'])

In [34]:
#filter by max time 
#max_time = max(gdfx['utc_time'])
#time_ref =  gdfx.loc[gdfx['utc_time'] ==  max_time,  ['nameofsample'] ].iloc[0,0]

### Visual : Simulate many points

In [35]:
#####################
#Vizualise winners and losers 
#####################

#Vizualise centroids and buffers
fig, ax = plt.subplots(figsize=figsize)

#sample layers
visualize.plot_winner(gdfx.loc[gdfx['plz_score'] ==  best,  : ] , 'sample_point')
visualize.plot_samples_points(gdfx.loc[gdfx['plz_score'] !=  best,  : ], 'sample_point')

mplleaflet.display(fig=fig)

In [36]:
gdfx.loc[gdfx['plz_score'] ==  best,  : ]

,nameofsample,plz,einwohner,id,plz_score,utc_time,country,city,buffer_size,relative_comp_quality,algorithm,dataset,sample_point,sample_buffer,Tot_Population,Num_districts,Code,All ages,Code_score
485,0,7.0,145571.0,53,29494.0,2022-01-05-10:40:14,DE,Berlin,0.02,random,simple,/content/drive/MyDrive/Colab Notebooks/data/GI...,POINT (13.50547 52.55331),"POLYGON ((13.52547 52.55331, 13.52395 52.54566...",145571.0,7,NaN,NaN,NaN


In [37]:
#plz is the number of district included
#einvohner is the total population
#id is the number of competitors included in the study area
